# Máster en Python Avanzado por Asociación AEPI

## Módulo III

### Mongo DB

MongoDB es una base de datos NoSQL orientada a documentos. Es una de las bases de datos más utilizadas en la industria debido a su flexibilidad y facilidad de uso. La arquitectura de MongoDB consiste en colecciones y documentos. MongoDB almacena los datos en formato de documento JSON, es decir, almacena datos en pares clave-valor.

Un grupo de documentos JSON se puede denominar colección. A diferencia de las tablas SQL, los documentos de MongoDB no tienen ningún esquema fijo. Con el uso del esquema dinámico, podemos realizar cambios fácilmente en la aplicación sin interrupciones.

Cómo realizar operaciones CRUD en MongoDB usando Python.

Para conectarnos con el servidor MongoDB usando python, necesitamos instalar un controlador de python llamado pymongo. Contiene herramientas que se utilizan para interactuar con MongoDB usando python.

Debemos crearnos un nuevo proyecto en Pycharm, usando un nuevo directorio virtual de virtualenv, a continuación para instalar pymongo, debemos escribir el siguiente comando en el símbolo del sistema.

```
pip install pymongo
```

Usaremos Compass para MongoDB, una GUI utilizada para interactuar con los datos almacenados en MongoDB, una vez que haya instalado correctamente Compass, conéctese a MongoDB pasando la siguiente cadena como entrada: mongodb://localhost:27017/ y haga clic en el botón de conexión.


A continuación, pulsaremos sobre Create database. En la ventana que se nos abre, pondremos, como nombre de la base de datos: tienda y como colección: productos.


Conectar a MongoDB desde Python: obtener base de datos

Para crear una nueva conexión, instanciamos a MongoClient pasándole las credenciales y los datos necesarios como el host, puerto y nombre de la base de datos. Hay otras maneras, pero esta es la que recomiendo porque nos permite cambiar fácilmente cualquier dato.


Todo eso podemos encerrarlo en una función como la que se ve a continuación:


In [ ]:
from pymongo import MongoClient

def obtener_bd():
    host = "localhost"
    puerto = "27017"
    base_de_datos = "tienda"
    cliente = MongoClient("mongodb://{}:{}".format(host, puerto))
    return cliente[base_de_datos]



El modelo

Vamos a usar una clase para simplificar las cosas, esta clase representará a un producto en nuestra base de datos, para ello crearemos un nuevo archivo de Python llamado producto.py


In [ ]:
class Producto:
    def __init__(self, nombre, precio, cantidad):
        self.nombre = nombre
        self.precio = precio
        self.cantidad = cantidad


Esto nos permitirá crear objetos de tipo Producto para pasarlos a los siguientes métodos de insertar y actualizar. No es obligatorio, pero así lo haremos.

Insertar o crear datos en MongoDB desde Python

Comencemos por insertar datos; de nuevo lo digo, lo encerramos en una función que recibe un objeto de tipo producto.

Para insertar un dato, primero obtenemos la colección que se encuentra dentro de nuestra base de datos, y llamamos al método insertar. A ese método le pasamos un diccionario con las claves y valores que queremos insertar.


In [ ]:
def insertar(producto):
    base_de_datos = obtener_bd()
    productos = base_de_datos.productos
    return productos.insert_one({
        "nombre": producto.nombre,
        "precio": producto.precio,
        "cantidad": producto.cantidad,
    }).inserted_id

Lo que devuelve nuestra función es el id insertado; no estamos obligados a hacerlo, pero permite saber si realmente se insertó o no.

Obtener datos de una colección de MongoDB a través de Python

Para esto, las colecciones nos dan un método o función llamada find, así como se hace normalmente en la CLI de MongoDB.

Si no le pasamos un criterio de búsqueda, devolverá todo lo existente. Más tarde, a esto podemos iterarlo con un ciclo foreach, pero primero veamos a la función:



In [ ]:
def obtener():
    base_de_datos = obtener_bd()
    return base_de_datos.productos.find()


Cuando llamamos al método obtener, podemos iterar los productos. Cabe mencionar que accedemos a sus propiedades a través de producto[clave], por ejemplo producto["precio"] en lugar de producto.precio.

Actualizar dato de MongoDB con PyMongo

Para actualizar un valor, llamamos al método update_one que recibe dos argumentos: el criterio de búsqueda (es decir, la condición que deben cumplir los datos para que se actualicen) y el nuevo valor.

El código queda así:


In [ ]:
from bson.objectid import ObjectId

def actualizar(id, producto):
    base_de_datos = obtener_bd()
    resultado = base_de_datos.productos.update_one(
        {
            '_id': ObjectId(id)
        },
        {
            '$set': {
                "nombre": producto.nombre,
                "precio": producto.precio,
                "cantidad": producto.cantidad,
            }
        })
    return resultado.modified_count


Recibe el id del producto que se va a actualizar, y un producto que tiene los nuevos datos, o sea, los que se pondrán al actualizar.
Lo que regresa la función es el número de documentos actualizados; si no cambiamos ningún valor entonces devolverá cero, porque no hubo cambios. Para saber por qué usamos ObjectId o $set sigue leyendo.

Aquí hay que ver algo muy importante, y es que como segundo argumento recibe el documento que será el remplazo del anterior. Si solamente queremos actualizar algunos datos, usamos el selector $set para establecer las propiedades necesarias, en lugar de remplazar todo el documento.

Al usar $set, si no indicamos una nueva propiedad o nuevo valor, los demás valores se quedan intactos. Si quieres remplazar el documento, entonces simplemente manda el diccionario en lugar de usar un selector de tipo $set.


El ObjectId

Aunque el ID es una cadena aleatoria, para hacer búsquedas a partir del mismo, debemos crear un objeto de tipo ObjectId, pasándole a la cadena que tiene el id como argumento. Esto es necesario siempre que filtremos por ID, ya sea para eliminar, actualizar, buscar o listar.


Eliminar datos de una colección en MongoDB

Ahora veamos el método que elimina, es más simple pues sólo recibe un id del producto que vamos a eliminar. Queda así:


In [ ]:
from bson.objectid import ObjectId

def eliminar(id):
    base_de_datos = obtener_bd()
    resultado = base_de_datos.productos.delete_one(
        {
            '_id': ObjectId(id)
        })
    return resultado.deleted_count


Usamos de nuevo el ObjectId para el criterio de selección. Lo que devuelve la función es el número de documentos eliminados.



### XML

Los archivos CSV son muy convenientes cuando se pretende guardar datos que compartan la misma estructura y no tengan jerarquías definidas. Cuan do se quiere guardar datos más complejos y con jerarquías, se usan otros tipos de formatos. Uno de los más populares es XML (extensive Markup Language; lenguaje de marcado extensible).

El formato XML permite definir jerarquías en los datos de manera independiente respecto al sistema en el que se estén utilizando, para así poder intercambiar información entre sistemas diferentes con el mismo fichero. Los ficheros en formato XML tienen dos componentes principales: las etiquetas y los atributos. Las etiquetas forman los bloques de datos y pueden contener (o no) atributos.

A continuación, se muestra un ejemplo de cómo se podrían modelar los datos de los planetas en formato XML:


In [ ]:
<sistemasolar>
    <planeta nombre="Mercurio">
        <masa>3.303e23</masa>
        <radio unidad="km">2.4397e6</radio>
    </planeta>
    <planeta nombre="Venus">
        <masa>4.869e+24</masa>
        <radio unidad="km">6.051e6</radio>
    </planeta>
    <planeta nombre="Tierra">
        <masa>5.976e+24</masa>
        <radio unidad="km">6.37814e6</radio>
    </planeta>
    <planeta nombre="Marte">
        <masa>6.421e+23</masa>
        <radio unidad="km">3.397e6</radio>
    </planeta>
    <planeta nombre="Jupiter">
        <masa>1.9e+27</masa>
        <radio unidad="km">7.1492e7</radio>
    </planeta>
    <planeta nombre="Saturno">
        <masa>5.688e+26</masa>
        <radio unidad="km">6.0268e7</radio>
    </planeta>
    <planeta nombre="Urano">
        <masa>8.686e+25</masa>
        <radio unidad="km">2.5559e7</radio>
    </planeta>
    <planeta nombre="Neptuno">
        <masa>1.024e+26</masa>
        <radio unidad="km">2.4746e7</radio>
    </planeta>
</sistemasolar>


SyntaxError: invalid syntax (3384022365.py, line 1)

Como se puede ver, la información se presenta de forma clara y concisa, y a simple vista se puede ver la jerarquía de la información que representa. La colección completa se llama sistema solar, la cual se define con las etiquetas `<sistema solar></sistema solar>`. Dentro de estas etiquetas se encuentra la definición de cada planeta, en la que cada nombre está definido como un atributo en la etiqueta principal (queda como `<planeta nombre="<nombre>"></planeta>`) y la descripción de cada planeta se encuentra ubicada dentro de cada bloque de planeta siguiendo el mismo patrón.

Los ficheros XML se utilizan para guardar información, pero también para ser intercambiados entre diferentes sistemas de bases de datos o aplicaciones. Para ello, Python provee herramientas de manejo de este tipo de ficheros.

La librería estándar para trabajar con ficheros XML es xml.etree.Ele mentTree y se suele importar como ET para abreviar. Las funciones más utilizadas para el manejo de este tipo de ficheros son las siguientes:

ET.parse(source, parser-None): devuelve un objeto tipo ElementTree extraído del parámetro source que puede ser la ruta de un fichero o un objeto tipo file. Además, se puede modificar el parseador que se utilizará configurando el parámetro parser.

ET.SubElement(parent, tag, attrib-{}, **extra): permite crear subelementos que añadir a parent. El parámetro tag será el nombre del elemento y attrib se utilizará para definir los atributos que deberá tener. Los parámetros añadidos como clave y valor en extra se añadirán como atributos extra.

ET.fromstring(text, parser-None): permite parsear la cadena de caracteres que se pasa en el parámetro text en un Element. Opcionalmente, se puede utilizar otro parser que no sea el objeto tipo estándar XMLParser.

class ET.Element(tag, attrib-{}, **extra): representa un elemento XML que contiene el nombre de la etiqueta proporcionado por tag y los atributos añadidos tanto en attrib como en extra haciendo uso de variables clave-valor. A continuación, se explican las funciones y los atributos más comunes disponibles:

•	attrib: es un diccionario que contiene todos los atributos del elemento. tag: representa el nombre que identifica al elemento.

•	get(key, default-None): permite devolver el valor del atributo con nombre key que tenga el elemento. Si no se encuentra, se devuelve el valor en el parámetro default.

•	find(match, namespaces=None): encuentra el primer subele mento cuyo nombre (tag) sea igual que match. Si match es una ruta, devuelve el primer elemento en esa ruta. Se pueden usar na mespaces para filtrar la búsqueda a solo un espacio de nombres específico de los que haya en los subelementos.

•	findall (match, namespaces=None): similar a find, pero una lista de subelementos. o devuelve

•	findtext(match, default=None, namespace=None): busca en el elemento y los subelementos hasta que encuentre uno que sea igual que match. Entonces, devuelve el texto asociado al elemento encontrado. El parámetro match puede ser el identificador del elemento o una ruta hasta el lugar donde encontrarlo.

•	iterfind(match, namespaces=None): similar a findall, pero devuelve un iterador en vez de una lista completa.

•	itertext(): crea un iterador que va devolviendo todos los textos internos de este elemento y de los subelementos.

class ET.ElementTree (element-None, file=None).

find, findall, findtext, iterfind: son iguales que las funciones apli cadas en Element, pero aplicadas a un Element Tree.

getroot(): devuelve el elemento raíz del árbol construido.

write(file, encoding="us-ascii", xml_declaration-None, default_namespace=None, method="xml", *, short_emp ty_elements=True): permite la escritura del árbol asociado en un fichero XML, pasado como parámetro utilizando el parámetro file. Los demás parámetros son descriptivos, pero cabe destacar que en method se puede especificar si el método debe ser xml, text o html.
A continuación, se muestra un ejemplo de cómo se pueden manipular archivos XML. Se desarrolla una función que extrae la información itera por cada nodo de los planetas y crea objetos tipo Planeta:


In [ ]:
from lxml.etree import ElementTree as ET

class Planeta:
    def __init__(self, nombre, masa, radio):
        self.nombre = nombre
        self.masa = masa
        self.radio = radio


def extraer_planetas_de_xml(fichero_planetas):
    planetas = []
    xml = ET.parse(fichero_planetas)
    planetas_xml = xml.findall('planeta')
    for planeta_xml in planetas_xml:
        planetas.append(extraer_planeta_de_xml(planeta_xml))
    return planetas


def extraer_planeta_de_xml(planeta_xml):
    nombre = planeta_xml.attrib['nombre']
    masa = planeta_xml.find('masa').text
    radio = planeta_xml.find('radio').text
    return Planeta(nombre, masa, radio)


Como se puede ver en el ejemplo, con pocas líneas de código se puede realizar una lectura e importación de datos desde un fichero XML a objetos tipo Planeta de forma simple. Una vez los objetos están creados, no existe diferencia en su procedencia.

A continuación, se muestra cómo se puede exportar el contenido a un archivo XML desde los objetos Planeta ya creados.



In [ ]:
def escribir_planetas(planetas, fichero_salida):
    raiz = ET.Element('sistemasolar-completo')  # raiz del arbol final
    for planeta in planetas:
        planeta_xml = ET.Element('planeta', attrib=dict(nombre=planeta.nombre))
        masa_xml = ET.Element('masa')  # agregando cada elemento de cada planeta
        masa_xml.text = str(planeta.masa)
        radio_xml = ET.Element('radio', attrib=dict(unidad='km'))
        radio_xml.text = str(planeta.radio)
        planeta_xml.extend([masa_xml, radio_xml])
        raiz.append(planeta_xml)
    arbol = ET.ElementTree(raiz)  # creando un objeto arbol para poder guardar
    arbol.write(fichero_salida)  # guardando el arbol completo en un fichero


### JSON

JSON (Notación de objetos de JavaScript) es un formato extremadamente popular para la serialización de datos, dada su aplicación general y su ligereza, además de ser bastante amigable para los humanos. En particular, se usa ampliamente en el mundo del desarrollo web, donde probablemente encontrará objetos serializados en JSON que se envían desde las API REST, la configuración de aplicaciones o incluso el almacenamiento de datos simple.


Leer JSON desde un archivo con Python

Puede usar el método json.load() para leer un archivo que contiene un objeto JSON. Supongamos que tiene un archivo llamado person.json que contiene un objeto JSON.


In [ ]:
import json

with open('person.json', 'r') as f:
    data = json.load(f)

# Mostrando el contenido del json
print(data)

# Mostrando data de primer nivel
print(data["name"])

# Mostrando data de segundo nivel
for item in data["languages"]:
    print(item)


Escribir JSON en un archivo con Python

Para escribir JSON en un archivo en Python, podemos usar el método json.dump().


In [ ]:
person_dict = {"name": "John", "surname": "Smith"}
def escribir_json():
    with open('person.txt', 'w') as json_file:
        json.dump(person_dict, json_file)


En el programa anterior, hemos abierto un archivo llamado person.txt en modo de escritura usando 'w'. Si el archivo aún no existe, se creará. Luego, se transforma person_dict en una cadena JSON que se guardará en el archivo person.txt mediante la función json.dump()

Cuando ejecute el programa, se creará el archivo person.txt.

Para analizar y depurar datos JSON, es posible que necesitemos imprimirlos en un formato más legible. Esto se puede hacer pasando parámetros adicionales al método json.dumps():


In [ ]:

def json_formateado():
    with open('person.json', 'r') as f:
        data = json.load(f)
    print(json.dumps(data, indent=4, sort_keys=True))


### Funciones de Hash


Una función hash es una función que dada una entrada de longitud variable, devuelve una secuencia de longitud fija, con algunas propiedades interesantes. Si por ejemplo aplicamos como entrada la cadena El Libro De Python, la salida será la siguiente.


In [ ]:
import hashlib

salida = hashlib.sha256(b"El Libro De Python").hexdigest()
print(salida)

Existen diferentes funciones hash, y en el caso anterior hemos usado la sha256. Una función hash se puede ver como una función resumen, ya que nos permite “resumir” un conjunto de datos de longitud variable en una secuencia de longitud fija (y relativamente corta). Podríamos también meter el libro entero de El Quijote un su función hash sha256 sería:

```
03f22ee1408a1bea9a7a9dfc0431051432c26a8a16fa6925d5246ff3235de3a4
```

Hemos por tanto resumido cientos de páginas en una sola línea.

A continuación, veremos:

•	Las propiedades de las funciones hash.

•	Las aplicaciones de las funciones hash.

•	Los diferentes tipos de funciones hash.

•	Cómo usar funciones hash en Python con hashlib.


Propiedades de las Funciones Hash

Las funciones hash tienen unas propiedades que las hacen muy útiles en el mundo de la criptografía y blockchain:

•	A pesar de que la entrada tiene una longitud arbitraria, la salida tiene una longitud fija. Esta longitud vendrá determinada por el tipo de función hash que se use. Por ejemplo, la sha256 devuelve siempre 256 bits (o 32 bytes).

•	Las funciones hash suelen ser rápidas de calcular.

•	Siendo x la entrada y hash(x) su función hash, es imposible (o muy muy difícil) obtener x a partir de hash(x). Es decir, que la función hash no es reversible. Si tenemos el hash de El Quijote, no podemos reconstruir el libro a partir del hash. Esto se conoce como resistencia a la primera preimagen.


•	Tiene que ser muy complicado (por no decir imposible) encontrar una nueva entrada x' siendo x'!=x tal que hash(x) = hash(x'). Es decir, tiene que ser imposible encontrar dos entradas cuya función hash sea la misma. Esto se conoce como resistencia a la segunda imagen.

•	Por último, la resistencia a colisiones o collision resistance nos dice que debe ser imposible encontrar dos entradas diferentes y distintas cuyo hash sea el mismo.

Es importante notar que algunas de estas características son de vital importancia, y que, si alguna de ellas dejara de cumplirse, el mundo de Internet estaría en serios problemas.

Por ejemplo, si se llegara a poder revertir una función hash, los pagos online que realizamos, contraseñas o incluso blockchains como bitcoin o ethereum podrían estar en problemas. Se dice que la computación cuántica podría romper las funciones hash, pero aún quedan años para eso.

Aplicaciones de Función Hash

Las funciones hash tienen aplicaciones en diferentes sectores. Explicamos a continuación sus casos de uso más relevantes:

•	Integridad de información: Podemos usar las funciones hash para asegurarnos de que un determinado contenido digital no ha sido modificado. Si por ejemplo calculamos el hash de un vídeo o un libro y lo almacenamos, tendremos una “huella digital” de dicho contenido. Si en un futuro nos envían ese mismo vídeo o libro, podemos calcular el hash otra vez y compararlo con el que teníamos almacenado anteriormente. Esto nos ahorra tener que ir fotograma a fotograma o página a página comparando ambos archivos.

•	Generar números aleatorios: Podemos usar las funciones hash para generar números aleatorios, o para ser más preciso para generar números pseudoaleatorios.

•	Firma digital: En la firma digital se suele firmar sólo el hash del mensaje en vez del contenido entero, lo que resulta más eficiente y reduce ciertos vectores de ataque.



•	Merkle Trees: Los merkle trees también pueden ser usados para resumir información, donde la misma es dividida en pequeños trozos y su hash es calculado recursivamente hasta obtener un único hash llamado merkle root. Estos son muy utilizados en la blockchain.

Tipos de Funciones Hash

Existen diferentes funciones hash, donde cada una tiene sus casos de uso. Algunas de las características más importantes son la longitud de la salida y el algoritmo que usan:

•	BLAKE: Tiene variantes como la BLAKE-2, BLAKE-3, siendo la última anunciada en 2020. Existen diferentes variantes en función del número de bits de su salida.

•	MD: Tiene múltiples variantes como la MD1, MD2, MD3, MD4 y MD5. El MD5 es muy usado para integridad de datos y fue introducido en la RFC 1321.

•	SHA: Tiene variantes como la SHA256, SHA512, SHA224, SHA384. El SHA256 es el usado por la criptomoneda Bitcoin.

•	KECCAK-256: Usado por la criptomoneda Ethereum


Funciones Hash en Python

Gracias a la librería hashlib de Python disponemos de prácticamente todas las funciones hash que existen. Veamos por ejemplo como usar la sha256.




In [ ]:
import hashlib

m = hashlib.sha256()
m.update(b"El Libro De Python")
salida = m.hexdigest()

print(salida)
# f7b5c532807800c540f5e4476ea1f6d968294fc34c90f2e7e64435ea3c054ce6


También podemos acceder al digest_size, es decir a la longitud de la salida. Este será un valor fijo dentro de cada función hash, y por ejemplo en el caso de sha256 es 32 bytes, o lo que viene siendo lo mismo, 256 bits. De ahí viene su nombre.



In [ ]:
print(m.digest_size)

# Salida
# 32
